In [17]:
# Load park coordinates

import pandas as pd
import geopandas as gpd

# Read the CSV using pandas
df = pd.read_csv('./dataset/parks_coordinates.csv')

# Convert 'latitude' and 'longitude' columns to numeric, forcing errors to NaN if any bad data
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Drop rows with missing or invalid coordinates, if needed
df = df.dropna(subset=['latitude', 'longitude'])

# Convert to GeoDataFrame
parks_df = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs='EPSG:4326'  # Assuming coordinates are in WGS84
)
print(parks_df[['name', 'geometry','states']].head())


                              name                geometry          states
0             Acadia National Park    POINT (-68.21 44.35)           Maine
1  National Park of American Samoa  POINT (-170.68 -14.25)  American Samoa
2             Arches National Park   POINT (-109.57 38.68)            Utah
3           Badlands National Park    POINT (-102.5 43.75)    South Dakota
4           Big Bend National Park   POINT (-103.25 29.25)           Texas


In [18]:
# Load states and time zone data
import geopandas as gpd
import pandas as pd
import random

# https://geodata.bts.gov/datasets/usdot%3A%3Atime-zones/about
# https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html

# Paths to shapefiles
STATES_PATH = './dataset/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp'
TIMEZONES_PATH = './dataset/NTAD_Time_Zones_467650596632424595/Time_Zones.shp'

# Exclusions
EXCLUDED_ZONES = {"Chamorro", "Samoa", "Atlantic"}
EXCLUDED_STATES = {
    "American Samoa", "Guam", "Commonwealth of the Northern Mariana Islands",
    "Puerto Rico", "United States Virgin Islands"}

# Load data
states = gpd.read_file(STATES_PATH)
time_zones = gpd.read_file(TIMEZONES_PATH)

# Ensure both layers use the same CRS
states = states.to_crs(time_zones.crs)

# Apply exclusions
time_zones_df = time_zones[~time_zones["zone"].isin(EXCLUDED_ZONES)]
states_df = states[~states["name"].isin(EXCLUDED_STATES)]

print(states_df[['name', 'geometry']].head())
print(time_zones_df[['zone', 'geometry']].head())

           name                                           geometry
0     Minnesota  POLYGON ((-89.95766 47.28691, -90.13175 47.292...
1       Montana  POLYGON ((-116.04823 49.00037, -113.0595 49.00...
2  North Dakota  POLYGON ((-97.22894 49.00089, -97.21414 48.902...
3        Hawaii  MULTIPOLYGON (((-155.93665 19.05939, -155.9080...
4         Idaho  POLYGON ((-116.04823 49.00037, -115.9678 47.95...
       zone                                           geometry
1   Eastern  MULTIPOLYGON (((-90.3253 46.71203, -89.98381 4...
2   Central  POLYGON ((-90.41012 46.57909, -90.40951 46.580...
3  Mountain  MULTIPOLYGON (((-114.36194 49.00112, -114.3484...
4   Pacific  MULTIPOLYGON (((-122.17237 49.00243, -122.1643...
5    Alaska  MULTIPOLYGON (((-155.75907 71.28518, -155.7562...


In [6]:
# input: staet row and timezone row, dirction('state' or 'zone', meaning finding relation from state to timezone or vice versa)
# output: list of relations between state and timezone

def calc_topology(state, timezone, direction):
    upper = 0.98
    lower = 0.02
    geom_state = state.geometry
    geom_other = timezone.geometry
    inter = geom_state.intersection(geom_other)
    ia = inter.area
    area_state = geom_state.area
    upper_area = upper * area_state
    lower_area = lower * area_state

    rels = []
    if ia == 0:
        if state['name'] == "Utah" and timezone.zone == "Pacific" :
            rels.append("touch")
        elif state['name'] == "Montana" and timezone.zone == "Central":
            rels.append("touch")
        else: rels.append("disjoint")
    if 0 < ia <= lower_area:
        rels.append("touch")
    if lower_area <ia <= upper_area:
        rels.append("overlaps") 
    if ia >= upper_area:
        special = {"Iowa", "Missouri", "Arkansas", "West Virginia"}
        if state['name'] in special:
            if(direction == "state"):
                rels.append("within")
            else:   
                rels.append("contains")
        elif state['name'] == "Alaska":
            if(direction == "zone"):
                rels.append("within")
            else:   
                rels.append("contains")
        else:
            if(direction == "state"):
                rels.append("covered by")
            else:   
                rels.append("covers")
    
    return sorted(set(rels))

In [7]:
# example usage
state_row = states_df[states_df['name'] == "Utah"].iloc[0]
time_zone_row = time_zones_df[time_zones_df['zone'] == "Pacific"].iloc[0]
calc_topology(state_row, time_zone_row, "state")

['touch']

In [8]:
import math

def calculate_bearing(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lat2 = math.radians(lat1), math.radians(lat2)
    d_lon = math.radians(lon2 - lon1)

    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - \
        math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)

    bearing_rad = math.atan2(x, y)
    bearing_deg = (math.degrees(bearing_rad) + 360) % 360
    return bearing_deg

def bearing_to_direction(bearing):
    """
    Convert bearing in degrees to 8-point compass direction.
    """
    directions = [
        'North', 'Northeast', 'East', 'Southeast',
        'South', 'Southwest', 'West', 'Northwest'
    ]
    idx = int(((bearing + 22.5) % 360) / 45)
    left_idx = (idx - 1) % len(directions)
    right_idx = (idx + 1) % len(directions)

    return [directions[left_idx], directions[idx], directions[right_idx]]

# input: row containing park1 and park2 names
# output: direction from park1 to park2
def deprecated_direction(parks_df,row):
    lat1 = parks_df.loc[parks_df['name'] == row['park1'], 'latitude'].iat[0]
    lon1 = parks_df.loc[parks_df['name'] == row['park1'], 'longitude'].iat[0]
    lat2 = parks_df.loc[parks_df['name'] == row['park2'], 'latitude'].iat[0]
    lon2 = parks_df.loc[parks_df['name'] == row['park2'], 'longitude'].iat[0]
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    return bearing_to_direction(bearing)


In [9]:
# The 8 compass points (exactly matching title-cased strings)
directions = [
    'North', 'Northeast', 'East', 'Southeast',
    'South', 'Southwest', 'West', 'Northwest'
]

def get_direction_from_coords(lat1, lon1, lat2, lon2):
    """
    Compare latitudes/longitudes to pick one of the 8 compass points.
    Returns one of: 'North', 'Northeast', 'East', 'Southeast', 
    'South', 'Southwest', 'West', 'Northwest', or 'Same location'.
    """
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Vertical component
    if d_lat > 0:
        vert = 'North'
    elif d_lat < 0:
        vert = 'South'
    else:
        vert = ''

    # Horizontal component
    if d_lon > 0:
        horiz = 'East'
    elif d_lon < 0:
        horiz = 'West'
    else:
        horiz = ''

    # Combine (and title-case so that e.g. "North" + "East" becomes "Northeast")
    if vert and horiz:
        return (vert + horiz).title()
    elif vert:
        return vert
    elif horiz:
        return horiz
    else:
        return 'Same location'

def direction_with_neighbors(direction):
    """
    Given one of the 8 compass strings, return [left_neighbor, direction, right_neighbor].
    If direction == 'Same location', return ['Same location'].
    """
    if direction == 'Same location':
        return [direction]

    idx = directions.index(direction)
    left_idx = (idx - 1) % len(directions)
    right_idx = (idx + 1) % len(directions)
    return [directions[left_idx], directions[idx], directions[right_idx]]

def calc_direction(parks_df, row):
    """
    Look up lat/lon for row['park1'] and row['park2']. If a name isn't found, raise ValueError.
    Otherwise, return [left_neighbor, direction, right_neighbor].
    """
    # Lookup for park1
    subset1 = parks_df.loc[parks_df['name'] == row['park1'], ['latitude', 'longitude']]
    if subset1.empty:
        raise ValueError(f"Park not found: '{row['park1']}'")
    lat1 = subset1['latitude'].iat[0]
    lon1 = subset1['longitude'].iat[0]

    # Lookup for park2
    subset2 = parks_df.loc[parks_df['name'] == row['park2'], ['latitude', 'longitude']]
    if subset2.empty:
        raise ValueError(f"Park not found: '{row['park2']}'")
    lat2 = subset2['latitude'].iat[0]
    lon2 = subset2['longitude'].iat[0]

    base_dir = get_direction_from_coords(lat1, lon1, lat2, lon2)
    return direction_with_neighbors(base_dir)


In [10]:
calc_direction(parks_df, {'park1': 'Indiana Dunes National Park', 'park2': 'Mammoth Cave National Park'})

['East', 'Southeast', 'South']

In [11]:
# calculate distance given lats and longs of two parks
def calc_park_to_park_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers

    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [12]:
# example usage
park1_lat = parks_df.loc[parks_df['name'] == 'Yosemite National Park', 'latitude'].iat[0]
park1_lon = parks_df.loc[parks_df['name'] == 'Yosemite National Park', 'longitude'].iat[0]
park2_lat = parks_df.loc[parks_df['name'] == 'Sequoia National Park', 'latitude'].iat[0]
park2_lon = parks_df.loc[parks_df['name'] == 'Sequoia National Park', 'longitude'].iat[0]

calc_park_to_park_distance(park1_lat, park1_lon, park2_lat, park2_lon)

171.80754378470672

In [13]:
import shapely.ops as ops

# calculate distance from park to state boundary
def calc_park_to_state_distance(park_name,parks_gdf, state_name, states_gdf) -> float:
    # Select park geometry
    park_row = parks_gdf[parks_gdf['name'] == park_name]
    if park_row.empty:
        raise ValueError(f"Park '{park_name}' not found in parks_gdf.")
    park_point = park_row.geometry.iloc[0]

    # Select state geometry
    state_row = states_gdf[states_gdf['name'] == state_name]
    if state_row.empty:
        raise ValueError(f"State '{state_name}' not found in states_gdf.")
    state_poly = state_row.geometry.iloc[0]

    # If park is inside state, distance is zero
    if state_poly.contains(park_point):
        return 0.0

    # Find nearest point on state boundary
    boundary = state_poly.boundary
    nearest_geom = ops.nearest_points(park_point, boundary)[1]

    # Compute haversine distance
    return calc_park_to_park_distance(
        park_point.y, park_point.x,
        nearest_geom.y, nearest_geom.x
    )

In [14]:
# example usage
calc_park_to_state_distance(
    'Yosemite National Park', parks_df, 'Alabama', states_df
)

2808.1730884567332